# Large Language Model Question and Answering Bot.

This will be based on the sentenceT5 by google 

Reference: https://github.com/amrrs/LLM-QA-Bot/blob/main/LLM_Q%26A_with_Open_Source_Hugging_Face_Models.ipynb 

In [1]:
!nvidia-smi

Thu Mar 16 20:42:04 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 531.18                 Driver Version: 531.18       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3070 Ti    WDDM | 00000000:2B:00.0  On |                  N/A |
| 39%   59C    P0               72W / 290W|    860MiB /  8192MiB |      2%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
! pip install pyproject.toml
! pip install -q langchain gpt-index llama-index transformers sentence_transformers


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: C:\Users\Jihoon.DESKTOP-1HIBMQO\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: C:\Users\Jihoon.DESKTOP-1HIBMQO\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
from llama_index import SimpleDirectoryReader, LangchainEmbedding, GPTListIndex,GPTSimpleVectorIndex, PromptHelper
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LLMPredictor
import torch
from langchain.llms.base import LLM
from transformers import pipeline

C:\Users\Jihoon.DESKTOP-1HIBMQO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
class FlanLLM(LLM):
    model_name = "google/flan-t5-base"
    pipeline = pipeline("text2text-generation", model=model_name, device=0, model_kwargs={"torch_dtype":torch.bfloat16})

    def _call(self, prompt, stop=None):
        return self.pipeline(prompt, max_length=9999)[0]["generated_text"]
 
    def _identifying_params(self):
        return {"name_of_model": self.model_name}

    def _llm_type(self):
        return "custom"


llm_predictor = LLMPredictor(llm=FlanLLM())

In [5]:
hfemb = HuggingFaceEmbeddings()
embed_model = LangchainEmbedding(hfemb)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cuda


## Import CTC product data 

In [6]:
import pandas as pd
import numpy as np
from llama_index import Document

In [7]:
import logging

logging.getLogger().setLevel(logging.CRITICAL)

In [8]:
product_detail_detail_path = "clean_data/cleaned_products_detailed.csv"
df_product_standard = pd.read_csv(product_detail_detail_path)
df_detailed = df_product_standard[['ctr_product_num','attr_value_en_sentence']]
df_detailed = df_detailed.dropna()
df_detailed = df_detailed.drop_duplicates()
df_detailed_subset = df_detailed.sample(frac=1)


In [9]:
path_standard = "Data\product\product_standard_attributes.csv" 

# put it in data farme
df_product_standard = pd.read_csv(path_standard, low_memory=False) #lazyway to solve low memory issue
df_right =df_product_standard[['ctr_product_num','merch_lob_nm','package_depth_qty','short_desc','package_height_qty','package_width_qty','package_volume_qty','package_weight_qty']]

df_right['ctr_product_num'] = pd.to_numeric(df_right['ctr_product_num'], errors='coerce')
df_right = df_right.dropna(subset=['ctr_product_num'])
df_right['ctr_product_num'] = df_right['ctr_product_num'].astype('int')
df_detailed_subset = df_detailed_subset.merge(df_right, on='ctr_product_num', how='left')

C:\Users\Jihoon.DESKTOP-1HIBMQO\AppData\Local\Temp\ipykernel_23432\271546728.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_right['ctr_product_num'] = pd.to_numeric(df_right['ctr_product_num'], errors='coerce')


In [ ]:
df_detailed_subset

In [11]:
#df_right =df_product_standard[['ctr_product_num','merch_lob_nm','package_depth_qty','short_desc','package_height_qty','package_width_qty','package_volume_qty','package_weight_qty']]


docLabels = []
# getting all the names of all json files
docLabels = [f for f in df_detailed_subset['package_depth_qty']]

sdecLabels = []
# getting all the names of all json files
sdecLabels = [f for f in df_detailed_subset['short_desc']]

subCatLabels = []
# getting all the names of all json files
subCatLabels = [f for f in df_detailed_subset['package_height_qty']]
widthLabels = [f for f in df_detailed_subset['package_width_qty']]
volumnLabels = [f for f in df_detailed_subset['package_volume_qty']]
weightLabels = [f for f in df_detailed_subset['package_weight_qty']]

In [12]:
data = []
j = 0
for i, row in df_detailed_subset.iterrows():
    #print(row['attr_value_en_sentence'])
    #data.append(row['attr_value_en_sentence'].split())
    output = 'package depth is ' + str(docLabels[j]) + '. ' + 'name of the product is ' + str(sdecLabels[j]) + '. ' + 'package height is ' + str(subCatLabels[j]) + '. '+'package width is ' + str(widthLabels[j]) + '. '+'package volumn is ' + str(volumnLabels[j]) + '. ' +'package weight is ' + str(weightLabels[j]) + '. ' + 'full description is ' + str(row['attr_value_en_sentence'] + '. ')
    data.append(output)
    j+=1

In [13]:
documents = [Document(t) for t in data]

In [14]:
test_documents = documents[:50]

In [15]:
# set number of output tokens
num_output = 250
# set maximum input size
max_input_size = 512
# set maximum chunk overlap
max_chunk_overlap = 20


prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap)

In [16]:
import logging

logging.getLogger().setLevel(logging.CRITICAL)

In [17]:
#index = GPTSimpleVectorIndex(documents, embed_model=embed_model, llm_predictor=llm_predictor)

index = GPTListIndex(test_documents, embed_model=embed_model, llm_predictor=llm_predictor)

#index.save_to_disk('index.json')

In [ ]:
test_documents

In [19]:
#response = index.query("product that is good for indoor") 
#print(response) 

In [23]:
response = index.query("how can be categorize the CLASS 2 HTCH 1-1/4?") 
print(response) 

TT+ ---------


In [24]:
response = index.query("what is TT+?") 
print(response) 

C:\Users\Jihoon.DESKTOP-1HIBMQO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\pipelines\base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Lights up and self inflates in secondsStakes and tethers includedPlugs in to standard outlet ------------


Cheery pick instance

In [ ]:
test_documents[-1]

In [ ]:
response = index.query("which product will be helpful for appearacbce if my indoor space?") 
print(response) 

In [ ]:
response = index.query("What kind of product is EC BOHEM BLK 6X9?") 
print(response) 

In [28]:
response = index.query("why is EC BOHEM BLK 6X9 non recyclable?") 
print(response) 

EC BOHEM BLK 6X9 is not recyclable


In [29]:
response = index.query("why is EC BOHEM BLK 6X9 is non recyclable?") 
print(response) 

a raised lip to trap and contain spills, dirt, and grease away from your vehicle's interiorFeatures a textured surface to help prevent cargo from shiftingMade in Turkey of high quality viscosePerfect area rug for medium to high traffic areasBlack modern/contemporary abstract indoor area rugRug is soft underfoot / to touch. ------------
